In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import networkx as nx
import pickle
import os
import sys
sys.path.insert(1, '../')
from utils import *

In [ ]:
plt.rcParams.update({'font.size': 15})

## Functions:

In [ ]:
def load_C_mean(filename):
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            C = np.array([float(i) for i in row])
    return C

In [ ]:
# cosine similarity between vectors
def similarity(v1,v2):
    
    return np.sum(v1*v2) / (np.sqrt((np.sum(v1*v1))*np.sum(v2*v2)))

In [ ]:
dataset = 'pr_school'

In [ ]:
weighted = True

In [ ]:
edgelist_name = '../data/pairwise/' + dataset + '_strength_edgelist_pairwise_sorted_norm.pkl'
with open(edgelist_name, 'rb') as f:
    edgelist = pickle.load(f)

nodes_list = np.unique(list(edgelist.keys()))

In [ ]:
nb_nodes = len(nodes_list)
#seed_size = int(round(nb_nodes/10))
seed_size = 1

# Comparison matrices

In [ ]:
mu = 0.1

Load all simple results

In [ ]:
mu = 0.1
betaT = 0
if dataset == 'hospital':
    beta_range = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    R0_range = [1.20,1.53,1.77,2,2.22,2.38,2.56,2.7]

elif dataset == 'high_school':
    beta_range = [0.5,0.6,0.7,0.8,0.9,1.0]
    R0_range = [1.25,1.40,1.55,1.71,1.83,1.88]

elif dataset == 'workplace':
    beta_range = [0.3,0.4,0.5,0.6,0.7,0.8]
    R0_range = [1.26,1.55,1.82,2.06,2.22,2.40]

elif dataset == 'conference':
    beta_range = [0.6,0.7,0.8,0.9,1]
    R0_range = [1.21,1.37,1.49,1.60,1.74]
    
elif dataset == 'pr_school':
    beta_range = [0.12,0.15,0.18,0.21,0.24,0.27,0.3,0.33,0.36]
    R0_range = [1.40,1.65,1.97,2.11,2.31,2.48,2.68,2.84,3.01]


In [ ]:
C_list_simple = []
for beta in beta_range:
    filename = "../Simple_model/code/results/inf_treeSIR/CL_mean_weighted_%s_beta_%.3f_betaT_0.000_mu_%.2f_init_%d.csv"%(dataset,beta,mu,seed_size)
    C = load_C_mean(filename)
    C_list_simple.append(C)

In [ ]:
len(C_list_simple)

Load all threshold results

In [ ]:
thresholds = [0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.22]
if dataset == 'pr_school':
    thresholds = [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16]

mu = 0.1

In [ ]:
C_list_thresh = []
for theta in thresholds:
    filename = "../Threshold_model/code/results/inf_treeSIR/C_mean_weighted_%s_th_%.2f_mu_%.3f.csv"%(dataset,theta,mu)
    C = load_C_mean(filename)
    C_list_thresh.append(C)

In [ ]:
len(C_list_thresh)

Compute similarity and put in matrix sim$\\$ (each row is a different $\theta$, each column a different $\beta_|$)

In [ ]:
sim = np.full((len(thresholds),len(beta_range)),0.0)
for i in range(len(thresholds)):
    for j in range(len(beta_range)): 
        #print('riga',i,'theta',thresholds[i],'colonna',j,'R0',R0_range[j])
        sim[i,j] = similarity(C_list_thresh[i],C_list_simple[j])

In [ ]:
folder = '../figs/'
if not os.path.exists(folder):
    os.makedirs(folder)

In [ ]:
plot_sim_different_axes(sim, 
                        R0_range, 
                        thresholds, 
                        '$R_0$', 
                        r'$\theta$',
                        folder + 'sim_threshold_simple_%s_mu_%.3f.pdf'%(dataset,mu),
                        save=True)